## FEMA Disasters/ Tornado code


by Jacqueline McBean-Blake

This application reads data from the tornado file.
Then extracts necessary columns, transforms values as needed and loads into tornado table. 
Finally via SQL code, it loads common fields into NATURAL_DISASTER table for easy comparison with other source data. Original source: https://www.spc.noaa.gov/wcm/data/1950-2018_actual_tornadoes.csv

In [33]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [34]:
csv_file = "tornadorawdata.csv"

In [35]:
#csv_file = os.path.join("Resources","tornadorawdata.csv")
tornadoraw_data_df = pd.read_csv(csv_file)
tornadoraw_data_df.head()


,om,yr,mo,dy,date,time,tz,st,stf,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,1,...,9.5,150,2,0,1,0,0,0,0,0
1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,2,...,3.6,130,1,1,1,135,0,0,0,0
2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,1,...,0.1,10,1,1,1,161,0,0,0,0
3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,1,...,0.6,17,1,1,1,113,0,0,0,0
4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,2,...,2.3,300,1,1,1,93,0,0,0,0


In [36]:
tornadoraw_data_df['id'] = tornadoraw_data_df.index

In [37]:
tornadoraw_data_df['data_source'] = 'TORNADO'

In [38]:
tornadoraw_data_df.head()

,om,yr,mo,dy,date,time,tz,st,stf,stn,...,ns,sn,sg,f1,f2,f3,f4,fc,id,data_source
0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,1,...,2,0,1,0,0,0,0,0,0,TORNADO
1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,2,...,1,1,1,135,0,0,0,0,1,TORNADO
2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,1,...,1,1,1,161,0,0,0,0,2,TORNADO
3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,1,...,1,1,1,113,0,0,0,0,3,TORNADO
4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,2,...,1,1,1,93,0,0,0,0,4,TORNADO


In [46]:
mytable_df=tornadoraw_data_df[['id','om', 'data_source','yr', 'mo', 'dy', 'date', 'time', 'tz', 'st', 'stf', 'stn', 'mag',
       'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon', 'len',
       'wid', 'ns', 'sn', 'sg', 'f1']]

In [47]:
mytable_df.head()

,id,om,data_source,yr,mo,dy,date,time,tz,st,...,slat,slon,elat,elon,len,wid,ns,sn,sg,f1
0,0,1,TORNADO,1950,1,3,1/3/1950,11:00:00,3,MO,...,38.77,-90.22,38.83,-90.03,9.5,150,2,0,1,0
1,1,2,TORNADO,1950,1,3,1/3/1950,11:55:00,3,IL,...,39.10,-89.30,39.12,-89.23,3.6,130,1,1,1,135
2,2,3,TORNADO,1950,1,3,1/3/1950,16:00:00,3,OH,...,40.88,-84.58,0.00,0.00,0.1,10,1,1,1,161
3,3,4,TORNADO,1950,1,13,1/13/1950,5:25:00,3,AR,...,34.40,-94.37,0.00,0.00,0.6,17,1,1,1,113
4,4,5,TORNADO,1950,1,25,1/25/1950,19:30:00,3,MO,...,37.60,-90.68,37.63,-90.65,2.3,300,1,1,1,93


In [48]:
rds_connection_string = "postgres:[ENTER your password ]@@localhost:5432/fema_disasters_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [49]:
engine.table_names()

['location',
 'fema',
 'earthquake',
 'data_source',
 'wild_fire',
 'natural_disaster',
 'tornado']

In [51]:
mytable_df.to_sql(name='tornado', con=engine, if_exists='append', index=False)

In [9]:
new_tornadoraw_data_df = tornadoraw_data_df[['om', 'date', 'st']].copy()
new_tornadoraw_data_df.head()

,om,date,st
0,1,1/3/1950,MO
1,2,1/3/1950,IL
2,3,1/3/1950,OH
3,4,1/13/1950,AR
4,5,1/25/1950,MO
